<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/deep%20learning/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Aprendizado por Reforço com Q-Learning

O **Aprendizado por Reforço (Reinforcement Learning - RL)** é uma área do Machine Learning onde um **agente** aprende a tomar **ações** em um **ambiente** para maximizar uma noção de **recompensa** cumulativa.

Diferente de outros tipos de aprendizado, não há um "gabarito". O agente aprende por tentativa e erro. O ciclo básico é:
1.  O agente observa o **estado** atual do ambiente.
2.  Com base no estado, o agente escolhe uma **ação**.
3.  O ambiente transita para um novo estado e dá ao agente uma **recompensa** (positiva ou negativa).
4.  O agente usa essa recompensa para atualizar sua estratégia (política) e repete o processo.

Neste notebook, vamos implementar um dos algoritmos mais famosos de RL, o **Q-Learning**, para ensinar um agente a resolver o ambiente "FrozenLake" da biblioteca `Gymnasium`.

In [ ]:
# Gymnasium é um toolkit padrão para desenvolver e comparar algoritmos de RL.
# Pygame é necessário para a renderização do ambiente.
# Se não os tiver, descomente e execute a linha abaixo.
# !pip install gymnasium pygame

import gymnasium as gym
import numpy as np
import random
import time
from IPython.display import clear_output

print("Bibliotecas importadas com sucesso!")


In [ ]:
"""
O ambiente `FrozenLake` é um mundo em grade 4x4. O objetivo é navegar do estado inicial (S) até o estado final (G) sem cair nos buracos (H).

- S: Start (Início)
- F: Frozen (Gelo, seguro para andar)
- H: Hole (Buraco, fim de jogo, sem recompensa)
- G: Goal (Objetivo, fim de jogo, com recompensa)

Vamos usar `is_slippery=False` para tornar o ambiente determinístico, ou seja, se o agente escolher ir para a direita, ele irá para a direita.
"""
env = gym.make("FrozenLake-v1", is_slippery=False)
# Para ver o agente em ação no final, use:
# env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="human")


# Obtendo o tamanho do espaço de estados e ações
state_space_size = env.observation_space.n
action_space_size = env.action_space.n

print(f"Número de Estados: {state_space_size}")
print(f"Número de Ações: {action_space_size}")



In [ ]:
"""
O Q-Learning usa uma **Tabela Q (Q-Table)** para aprender. A tabela tem uma linha para cada estado e uma coluna para cada ação. O valor `Q(s, a)` representa a "qualidade" (recompensa futura esperada) de se tomar a ação `a` no estado `s`.

A regra de atualização da Tabela Q é:
`Q(s, a) <-- Q(s, a) + alpha * [recompensa + gamma * max(Q(s', a')) - Q(s, a)]`
"""

# Inicializando a Tabela Q com zeros
q_table = np.zeros((state_space_size, action_space_size))

# Hiperparâmetros do algoritmo
total_episodes = 10000        # Número de jogos que o agente vai jogar
learning_rate = 0.1           # Taxa de aprendizado (alpha)
discount_factor = 0.99        # Fator de desconto (gamma)

# Parâmetros para a estratégia Epsilon-Greedy (Exploration vs Exploitation)
epsilon = 1.0                 # Taxa de exploração inicial
max_epsilon = 1.0             # Valor máximo de epsilon
min_epsilon = 0.01            # Valor mínimo de epsilon
epsilon_decay_rate = 0.0005   # Taxa de decaimento de epsilon

print("Tabela Q e hiperparâmetros inicializados.")
print("Formato da Tabela Q:", q_table.shape)



In [ ]:
print("\nIniciando o treinamento...")
time.sleep(1)

# Loop principal de treinamento
for episode in range(total_episodes):
    # Resetar o ambiente para um novo episódio
    state, info = env.reset()
    terminated = False

    while not terminated:
        # Dilema Exploração vs. Exploitation (Epsilon-Greedy)
        if random.uniform(0, 1) < epsilon:
            # Escolher uma ação aleatória (Exploração)
            action = env.action_space.sample()
        else:
            # Escolher a melhor ação conhecida (Exploitation)
            action = np.argmax(q_table[state, :])

        # Executar a ação no ambiente
        new_state, reward, terminated, truncated, info = env.step(action)

        # Atualizar a Tabela Q com a fórmula do Q-Learning
        # O agente caiu no buraco ou chegou ao objetivo
        if terminated or truncated:
            q_table[state, action] = q_table[state, action] + learning_rate * (reward - q_table[state, action])
        else:
            q_table[state, action] = q_table[state, action] + learning_rate * (
                reward + discount_factor * np.max(q_table[new_state, :]) - q_table[state, action]
            )

        # Atualizar o estado
        state = new_state

    # Decaimento do Epsilon (para reduzir a exploração ao longo do tempo)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate * episode)

    # Feedback visual do treinamento
    if (episode + 1) % 1000 == 0:
        clear_output(wait=True)
        print(f"Treinando... Episódio: {episode + 1}/{total_episodes}")

clear_output(wait=True)
print("Treinamento concluído!")


In [ ]:
"""
Após o treinamento, a Tabela Q contém a política ótima aprendida pelo agente. Para cada estado (linha), a coluna com o maior valor indica a melhor ação a ser tomada.
"""
print("Tabela Q Final (arredondada):")
print(q_table.round(3))


In [ ]:
"""
Agora, vamos ver o nosso agente em ação! Vamos executar vários episódios sem exploração (epsilon = 0) e ver com que frequência ele consegue atingir o objetivo.
"""
env.close() # Fechar o ambiente anterior para criar um novo com modo de renderização
eval_env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="human")

num_eval_episodes = 5
total_wins = 0

print("\nIniciando a avaliação do agente treinado...")

for episode in range(num_eval_episodes):
    state, info = eval_env.reset()
    terminated = False
    print(f"\n--- Episódio de Avaliação #{episode + 1} ---")
    time.sleep(1) # Pausa para visualização

    while not terminated:
        eval_env.render() # Mostra o estado atual do ambiente

        # Escolher a melhor ação da Tabela Q (sempre exploitation)
        action = np.argmax(q_table[state, :])

        new_state, reward, terminated, truncated, info = eval_env.step(action)
        state = new_state
        time.sleep(0.3)

    eval_env.render() # Renderizar o último estado
    if reward == 1.0:
        print("Agente alcançou o objetivo! (Vitória)")
        total_wins += 1
    else:
        print("Agente caiu em um buraco. (Derrota)")
    time.sleep(1.5)

eval_env.close()

# Calculando a taxa de sucesso
success_rate = (total_wins / num_eval_episodes) * 100
print(f"\nTaxa de sucesso na avaliação: {success_rate}% ({total_wins}/{num_eval_episodes} vitórias)")



In [ ]:
"""
### Conclusão

Neste notebook, implementamos com sucesso o algoritmo Q-Learning do zero para treinar um agente a resolver um problema clássico de RL.

**Principais aprendizados:**
1.  **Ciclo do RL:** Vimos na prática o ciclo de `estado -> ação -> recompensa` que fundamenta o aprendizado por reforço.
2.  **Tabela Q:** Entendemos como uma simples tabela pode armazenar o "conhecimento" do agente sobre o ambiente, mapeando a qualidade de cada par estado-ação.
3.  **Exploração vs. Exploitation:** Implementamos a estratégia epsilon-greedy, um conceito crucial em RL que equilibra a necessidade de explorar novas ações e de explorar as melhores ações já conhecidas.

O Q-Learning é poderoso, mas limitado a ambientes com espaços de estados e ações discretos e relativamente pequenos.
Para problemas mais complexos (como jogos com telas de pixels ou controle de robôs), a Tabela Q se torna inviável.
Nesses casos, algoritmos mais avançados como **Deep Q-Networks (DQN)** são usados, onde uma rede neural é treinada
 para aproximar a função Q, em vez de armazená-la em uma tabela.
"""